In [1]:
from dask.distributed import fire_and_forget
from dask.distributed import Client, Variable , Queue , Lock ,LocalCluster
from dask_jobqueue import SLURMCluster
from dask.distributed import  utils_perf
from dask.distributed import Client, LocalCluster
import dask
import redis
from bloom_filter2 import BloomFilter
import lzma
from dask import dataframe as dd

In [2]:
#!/usr/bin/env python
# coding: utf-8
import sys
sys.path.append('../../..')
#sys.path.append( '/home/cactuskid13/miniconda3/pkgs/')
print(sys.path)
import ete3
import random
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import itertools
import dask
import warnings
from dask import dataframe as dd
import pickle
path = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/'

['/work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/RDF_Graphnet/rdf2network', '/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/ML2/lib/python310.zip', '/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/ML2/lib/python3.10', '/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/ML2/lib/python3.10/lib-dynload', '', '/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/ML2/lib/python3.10/site-packages', '../../..']


In [3]:
from dask.distributed import fire_and_forget
from dask.distributed import Client, Variable , Queue , Lock ,LocalCluster
from dask_jobqueue import SLURMCluster
from dask.distributed import  utils_perf
from dask.distributed import Client, LocalCluster
import dask
import redis
from bloom_filter2 import BloomFilter
import lzma
from dask import dataframe as dd

#using distributed computation on a slurm cluster here. This is my particular config. You will need to alter this: https://distributed.dask.org/en/stable/
NCORE = 4
print('deploying cluster')
cluster = SLURMCluster(
    #change theses settings for your cluster
    walltime='4:00:00',
    n_workers = NCORE,
    cores=NCORE,
    processes = NCORE,
    interface='ib0',
    memory="20GB",
    #job_script_prologue=[
    #' source /work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/etc/profile.d/conda.sh  ' ,
    #'conda activate ML2'
    #],
    #scheduler_options={'interface': 'ens2f0' },
    #if gpu node
    scheduler_options={'interface': 'ens3f0np0' },
    #extra=[ "--lifetime-stagger", "4m"]
)
print(cluster.job_script())
print(cluster)
cluster.scale(jobs = 100)
print(cluster.dashboard_link)
client = Client(cluster , timeout='450s' , set_as_default=True )

deploying cluster
#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -n 1
#SBATCH --cpus-per-task=4
#SBATCH --mem=19G
#SBATCH -t 4:00:00

/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/ML2/bin/python -m distributed.cli.dask_worker tcp://10.203.101.82:42139 --nthreads 1 --nworkers 4 --memory-limit 4.66GiB --name dummy-name --nanny --death-timeout 60 --interface ib0

SLURMCluster(73602189, 'tcp://10.203.101.82:42139', workers=0, threads=0, memory=0 B)
http://10.203.101.82:8787/status


In [10]:
#find which species each of the cogs has an interaction in
link_df = dd.read_csv(path + 'datasets/STRING/protein.links.full.v11.5.txt',  blocksize=15e6 , header = 0, sep = ' ')
print(link_df)

Dask DataFrame Structure:
                   protein1 protein2 neighborhood neighborhood_transferred fusion cooccurence homology coexpression coexpression_transferred experiments experiments_transferred database database_transferred textmining textmining_transferred combined_score
npartitions=102701                                                                                                                                                                                                                                             
                     object   object        int64                    int64  int64       int64    int64        int64                    int64       int64                   int64    int64                int64      int64                  int64          int64
                        ...      ...          ...                      ...    ...         ...      ...          ...                      ...         ...                     ...      ...                  ...

In [ ]:
#compute bloom filters for protein pairs

import tqdm 

@dask.delayed
def protlinks_species( df ):
    df = df[~df['protein1'].isna()]
    df = df[~df['protein2'].isna()]
    df.protein1 = df.protein1.map(lambda x:str(x))
    df.protein2 = df.protein2.map(lambda x:str(x))
    df['protlinks'] = df.protein1 + '_' + df.protein2 
    ret = set(df.protlinks.unique())
    return ret

@dask.delayed
def return_filter(protlinks, verbose = True):
    if type( protlinks ) == tuple:
        protlinks = protlinks[0]
    b=BloomFilter(max_elements=10**8, error_rate=0.001 ,start_fresh = True)
    for p in protlinks:
        b.add( p )
    retlen = len(protlinks)
    return   b , retlen

@dask.delayed
def sumfilter(f1,f2, total ):
    if type( f1 ) == tuple:
        f1 = f1[0]
    if type( f2 ) == tuple:
        f2 = f2[0]
    f3 = f1.__ior__(f2)
    return f3 , total

def treesum(totalfilter):
    #print(len(totalfilter))
    while len(totalfilter)>1:
        next_round= []
        for i in range(0,len(totalfilter),2):
            if i+1 < len(totalfilter):
                next_round.append( sumfilter( totalfilter[i][0] , totalfilter[i+1][0] , totalfilter[i][1]+totalfilter[i+1][1]  ) )
        if len(totalfilter) % 2 !=0:
            next_round.append(totalfilter[-1])
        totalfilter = next_round
        #print(len(totalfilter))
    return totalfilter

b=BloomFilter(max_elements=10**8, error_rate=0.001 ,start_fresh = True)
partitions  = link_df.to_delayed()
print('map cogs')
res1 = [ protlinks_species(p) for p in partitions ]
print('done')
print('make filters')
res2 = [ return_filter(p) for p in res1 ]


finals =[]

with tqdm.tqdm(total=int(len(res2)/512)+1) as pbar:
    for chunk in range(int(len(res2)/512)+1):
        #print(chunk*1024)
        res3 = res2[chunk*512:(chunk+1)*512]
        res4 = treesum(res3)
        res4 = dask.compute(res4)
        finals.append(res4[0])
        pbar.set_description('processed: %d' % (1 + chunk ))
        pbar.update(1)  

#dask.compute(*finals)

with open('bloomfinal_big.pkl' , 'wb' ) as finalout:
    finalout.write(pickle.dumps(finals))

map cogs
done
make filters


processed: 124:  62%|██████▏   | 124/201 [3:42:45<2:15:15, 105.40s/it]

In [ ]:
print(link_df[].compute())

In [ ]:
import functools
with open('bloomfinal_big.pkl' , 'rb' ) as finalout:
    resfinal = pickle.loads(finalout.read()) 
print(resfinal)

In [ ]:
print(1437758757 -100000000)

In [ ]:
def check_filters(element,filters):
    for f in filters:
        if element in f[0][0]:
            return True
    return False

In [ ]:
#check filters
bfilter = functools.partial(check_filters , filters= resfinal)

In [21]:
assert True == bfilter('')
assert True == bfilter('xxxxxxxxxxxxxxxxxxxxx')